In [2]:
import urllib2
import math
import re
from bs4 import BeautifulSoup
from pprint import pprint
from threading import Thread
from collections import deque
import time
from ua import UserAgent

Mozillia/5.0 (PPC; rv:1.0) Gecko/20050112 Firefox/1.0


In [3]:
class GoogleSearch:
    USER_AGENT = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8"
    SEARCH_URL = "https://google.com/search"
    RESULT_SELECTOR = ".srg h3.r a"
    TOTAL_SELECTOR = "#resultStats"
    RESULTS_PER_PAGE = 10
    DEFAULT_HEADERS = [
            ('User-Agent', USER_AGENT),
            ("Accept-Language", "en-US,en;q=0.5"),
        ]

    def search(self, query, num_results = 10, prefetch_pages = True, prefetch_threads = 10, language = "en"):
        searchResults = []
        pages = int(math.ceil(num_results / float(GoogleSearch.RESULTS_PER_PAGE)));
        fetcher_threads = deque([])
        total = None;
        ua = UserAgent()
        ua = ua.random()
        DEFAULT_HEADERS = [
            ('User-Agent', ua),
            ("Accept-Language", "en-US,en;q=0.5"),
        ]
        print ua
        print 'pg: ', pages
        for i in range(pages) :
            start = i * GoogleSearch.RESULTS_PER_PAGE
            print 'start: ', start 
            opener = urllib2.build_opener()
            opener.addheaders = GoogleSearch.DEFAULT_HEADERS
            response = opener.open(GoogleSearch.SEARCH_URL + "?q="+ urllib2.quote(query) + "&hl=" + language + ("" if start == 0 else ("&start=" + str(start))))
            soup = BeautifulSoup(response.read(), "lxml")
            response.close()
            if total is None:
                totalText = soup.select(GoogleSearch.TOTAL_SELECTOR)[0].children.next().encode('utf-8')
                print 'totalText: ', totalText
                total = long(re.sub("[',\. ]", "", re.search("(([0-9]+[',\. ])*[0-9]+)", totalText).group(1)))
                print 're-search: ', re.search("(([0-9]+[',\. ])*[0-9]+)", totalText).group(1)
                
            results = self.parseResults(soup.select(GoogleSearch.RESULT_SELECTOR))
            if len(searchResults) + len(results) > num_results:
                del results[num_results - len(searchResults):]
            searchResults += results
        return SearchResponse(searchResults, total);
    def parseResults(self, results):
        tStart = time.time()
        searchResults = [];
        for result in results:
            title = result.text
            searchResults.append(SearchResult(title))
        return searchResults

In [4]:
qstring1 = '是日本戰國時代的大名及江戶幕府第一任征夷大將軍，全名德川次郎 wiki'

In [22]:
rs = GoogleSearch().search(qstring, num_results = 7)

Mozilla/5.0 (PPC) AppleWebKit/502.0 (KHTML, like Gecko) Version/2.0305.362 Safari/502
pg:  1
start:  0
totalText:  About 396 results
re-search:  396


In [23]:
for i in range(len(rs.results)):
    print rs.results[i].title

拉丁语- 维基百科，自由的百科全书
Dreye權威釋義| 拉丁文 - Dr.eye 譯典通
lingua Latina 拉丁語 - 華人百科
埃涅阿斯紀- Wikiwand
古羅馬廣場- Wikiwand
勞古多羅方言- Wikiwand
狄奧多里克大帝- Wikiwand


In [21]:
qstring = '屬於印歐語系義大利語族。是最早在拉提姆地區（今義大利的拉齊奧 wiki'
response = GoogleSearch().search(qstring, num_results = 7)
print response.results[0].title

Mozilla/5.0 (PPC) AppleWebKit/562.0 (KHTML, like Gecko) Version/0.01028.257 Safari/562
pg:  1
start:  0
totalText:  About 396 results
re-search:  396
拉丁语- 维基百科，自由的百科全书


In [1]:
class SearchResponse:
    def __init__(self, results, total):
        self.results = results;
        self.total = total;

class SearchResult:
    def __init__(self, title):
        self.title = title
        self.__text = None
        self.__markup = None